# Preprocessing

## 1. Get a shp file of seoul road network

In [ ]:
import os
from os import listdir

import itertools
from tqdm import tqdm 

import numpy as np
import pandas as pd

import geopandas

### 1. 1. Load data and select service link

In [ ]:
# Check if the intersection of service link in speed data is the same number of links (5106)

for i, file in enumerate(listdir(os.getcwd()+'/seoul_speed_raw_data')):
    if i>=12: # Select data from 2022_1001_1010
        speed_data = pd.read_csv('./seoul_speed_raw_data/'+file)
        if i==12:
            print(f'{file} has initialized the link id list')
            link_id_list = speed_data['LINK_ID'].unique()
            link_id_list.sort()
            print(f'The number of link id in speed data is {len(link_id_list)}')
        else:
            new_link_id_list = speed_data['LINK_ID'].unique()
            new_link_id_list.sort()
            if not np.array_equal(link_id_list, new_link_id_list):
                print(f'{file} has different link id')
                print(len(link_id_list), len(new_link_id_list))
                print(np.setdiff1d(new_link_id_list, link_id_list))
                # Print which link is different
                print(len(np.intersect1d(link_id_list, new_link_id_list)))
                link_id_list = np.intersect1d(link_id_list, new_link_id_list)
                print('new link list created')
                
            else:
                print(f'{file} has same link id')
del file, i, speed_data, link_id_list, new_link_id_list

# 2022_1001_1010_speed_5min.txt has intersection of link id in speed data

In [ ]:
# Handmade metropolitan suburbs link shp file, September 5th, 2022
metropolitan_link = geopandas.read_file('[2022-09-05]NODELINKDATA/metropolitan_link.shp', encoding='utf-8')
metropolitan_link = metropolitan_link.apply(pd.to_numeric, errors='ignore')
metropolitan_link.sort_values(by=['LINK_ID'], inplace=True)
print(f'The length of metropolitan link data = {len(metropolitan_link)}')

metropolitan_node = geopandas.read_file('[2022-09-05]NODELINKDATA/metropolitan_node.shp', encoding='utf-8')
metropolitan_node = metropolitan_node.apply(pd.to_numeric, errors='ignore')
metropolitan_node.sort_values(by=['NODE_ID'], inplace=True)
print(f'The length of metropolitan node data = {len(metropolitan_node)} \n')

# Check the intersection of service link data -> Service link of 2021_0901_0910 is the intersection of whole data
load_service_link_list = pd.read_csv('seoul_speed_raw_data/2022_1001_1010_speed_5min.txt')
service_link_list_from_speed = load_service_link_list['LINK_ID'].unique()
del load_service_link_list
service_link_list_from_speed.sort()

# Utilization of standard link-service link mapping data in Seoul as of April 2022
seoul_link_mapping = pd.read_csv('서울시_표준링크_매핑정보_2022년4월_기준.csv')
print(f'The length of seoul mapping data = {len(seoul_link_mapping)}')
print(f"Number of uniuqe standard link id == {len(seoul_link_mapping['표준링크아이디'].unique())}")

if (len(seoul_link_mapping['표준링크아이디'].unique()) == len(seoul_link_mapping)):
    print(f'Every standard link id is unique \n')
else:
    print(f'Every standard link id is not unique \n')
    
print(f"Number of cases where [standard link id==service link id] in seoul mapping data = ",
      f"{sum(seoul_link_mapping['표준링크아이디'] == seoul_link_mapping['서비스링크'])}")
print(f"Number of cases where [standard link id.isna()] in seoul mapping data = ",
      f"{sum(seoul_link_mapping['표준링크아이디'].isna())}")
print(f"Number of cases where [service link id.isna()] in seoul mapping data = ",
      f"{sum(seoul_link_mapping['서비스링크'].isna())} \n")

A = seoul_link_mapping[
    ~(seoul_link_mapping['표준링크아이디'] == seoul_link_mapping['서비스링크']) &
    ~seoul_link_mapping['서비스링크'].isna()]['서비스링크'].unique()
B = seoul_link_mapping[
    ~(seoul_link_mapping['표준링크아이디'] == seoul_link_mapping['서비스링크']) &
    seoul_link_mapping['서비스링크'].isna()]['표준링크아이디'].unique()

print(f'{len(np.intersect1d(A, B))} data in A and B is an intersection. However, ignore this.')
del A, B

![Alt text](utils/seoul_mapping_relation.png)

In [ ]:
# Insert standard link id to service link where it is nan value
seoul_link_mapping.loc[seoul_link_mapping['서비스링크'].isna(), ['서비스링크']]=seoul_link_mapping[seoul_link_mapping['서비스링크'].isna()]['표준링크아이디']
seoul_link_mapping['서비스링크'] = seoul_link_mapping['서비스링크'].astype('int64')

# There are service links that exist only in the speed data and not in the mapping data.
not_in_mapping_list = list(set(service_link_list_from_speed) - set(seoul_link_mapping['서비스링크'].unique()))
print(f'Among {len(service_link_list_from_speed)} service links in speed data, {len(not_in_mapping_list)} are not in mapping data')

# Insert those data
add_to_list = metropolitan_link[metropolitan_link['LINK_ID'].isin(not_in_mapping_list)]['LINK_ID'].values
seoul_link_mapping = pd.concat([seoul_link_mapping, pd.DataFrame(np.vstack((add_to_list, add_to_list)).T, columns=seoul_link_mapping.columns)]).reset_index(drop=True)
del not_in_mapping_list, add_to_list
seoul_link_mapping = seoul_link_mapping[seoul_link_mapping['서비스링크'].isin(service_link_list_from_speed)]
del service_link_list_from_speed
seoul_link_mapping.sort_values(by=['서비스링크', '표준링크아이디'], inplace=True)
seoul_link_mapping.reset_index(drop=True, inplace=True)

# Set seoul data set
seoul_link = metropolitan_link[metropolitan_link['LINK_ID'].isin(seoul_link_mapping['표준링크아이디'].values)].copy()
seoul_link.sort_values(by=['LINK_ID'], inplace=True)
seoul_link.reset_index(drop=True, inplace=True)
seoul_link = pd.merge(seoul_link, seoul_link_mapping, how='inner', left_on=['LINK_ID'], right_on=['표준링크아이디'])
service_link_list = seoul_link['서비스링크'].unique()
service_link_list.sort()
print(f'The number of service link is {len(service_link_list)}')

### 1. 2. Check number of nan in links (cost time, do not run)

In [ ]:
# Used to check the number of nan
nan_counts = np.zeros(len(service_link_list))

for i, file in enumerate(listdir(os.getcwd()+'/seoul_speed_raw_data')):
    if i>=12:
        speed_data = pd.read_csv('./seoul_speed_raw_data/'+file)
        speed_data['PRCS_SPD'] = speed_data['PRCS_SPD'].astype('int64')
        speed_data['PRCS_SPD'].replace(0, np.nan, inplace=True)
        date_list = speed_data['PRCS_DAY'].unique()
        print(f'{file}')
        print(f'The length of day is {len(date_list)}')
        for idx, service_link in enumerate(service_link_list):
            has_value = (~speed_data[speed_data['LINK_ID']==service_link]['PRCS_SPD'].isna()).sum()
            if 12*24*len(date_list)-has_value!=0:
                nan_counts[idx]+=(12*24*len(date_list)-has_value)

for count, service_link in zip(nan_counts, service_link_list):
    if count/(24*12*61)>0.10:
        print(service_link, count/(24*12*61))

del count, file, has_value, idx, nan_counts, service_link, speed_data, date_list

# Missing more than 10% were excluded
print('print end')

### 1. 3 Exclude to many missing

In [ ]:
# Remove the missing links
nan_list = [1050048100, 1179910010, 1179920110, 2180000101, 2180000201, 2180002102, 2180002201]

mask = np.in1d(service_link_list, nan_list, invert=True)
service_link_list = service_link_list[mask]
print(f'The number of service links in service_link_list = {len(service_link_list)}')

seoul_link = seoul_link[~seoul_link['서비스링크'].isin(nan_list)]
seoul_link.sort_values(by=['LINK_ID'], inplace=True)
seoul_link.reset_index(drop=True, inplace=True)
del nan_list
seoul_link_restore = seoul_link.copy()

print(f'The number of links = {len(seoul_link)}')
print(f'The number of service links in seoul_link = {len(seoul_link["서비스링크"].unique())}')

crs = seoul_link.crs # Must store the original crs    

# Change name to avoid error
seoul_link = seoul_link.rename(columns={'서비스링크': 'SERVICE_ID','표준링크아이디': 'STD_ID'})
seoul_link_restore = seoul_link_restore.rename(columns={'서비스링크': 'SERVICE_ID','표준링크아이디': 'STD_ID'})


## 2. Reconnecting links

### 2. 1. Reconnect 2-hop missing

In [ ]:
# Skip this stage if data does not need to be restored
for iteration in range(2):
    seoul_node_list = np.union1d(np.unique(seoul_link_restore['F_NODE'].values), np.unique(seoul_link_restore['T_NODE'].values))

    # The link which is not contained in seoul_link_restore
    target_link = metropolitan_link[~metropolitan_link['LINK_ID'].isin(seoul_link_restore['LINK_ID'].values)]

    # But has F_NODE and T_NODE inside seoul link
    target_link = target_link[
        target_link['F_NODE'].isin(seoul_node_list) &
        target_link['T_NODE'].isin(seoul_node_list)]
    target_link.sort_values(by=['LINK_ID'])
    target_link.reset_index(drop=True, inplace=True)

    print(f'Before reconnecting the 1-hop missing, the length of seoul link is {len(seoul_link_restore)}')
    print(f'Number of candidate link {len(target_link)}')

    # Reconnect 1-hop per iteration
    for i in tqdm(range(len(target_link))):
        t_node = target_link.iloc[i]['T_NODE']
        f_node = target_link.iloc[i]['T_NODE']
        link_id_div = target_link.iloc[i]['LINK_ID']/1000
        matches = seoul_link_restore[
            (seoul_link_restore['F_NODE'] == t_node) &
            (~(seoul_link_restore['T_NODE'] == f_node)) &
            (seoul_link_restore['LINK_ID']/1000 == link_id_div)]

        if len(matches)==0:
            matches = seoul_link_restore[
                (seoul_link_restore['F_NODE'] == t_node) &
                (~(seoul_link_restore['T_NODE'] == f_node))]

        added_link = target_link.iloc[i].copy()
        if len(matches)>0:
            matches.reset_index(drop=True, inplace=True)
            added_link['SERVICE_ID'] = matches.iloc[0]['SERVICE_ID']
            added_link['STD_ID'] = added_link['LINK_ID']

            seoul_link_restore = pd.concat([seoul_link_restore, added_link.to_frame().transpose()])

    seoul_link_restore.reset_index(drop=True, inplace=True)
    print(f'After reconnecting the 1-hop missing, the length of seoul link is {len(seoul_link_restore)}')
del t_node, f_node, link_id_div, matches, added_link, iteration, 
seoul_link_restore.crs = crs

### 2.2 Multi-hop reconnecting of major links

In [ ]:
print(f'Before reconnecting the multi-hop missing, the length of seoul link is {len(seoul_link_restore)}')
for iteration in range(3):
    target_link = metropolitan_link[
        (~metropolitan_link['LINK_ID'].isin(seoul_link_restore['LINK_ID'].values)) &
        (metropolitan_link['ROAD_NAME'].isin([
            '강변북로', '올림픽대로', '동부간선도로', '강남대로', '경부고속도로', '선유로', '노들로', '강변역로'
            '청파로', '성산로', '원효로', '내부순환로', '강동대로', '반포대로', '녹사평대로', '여의동로', '여의서로',
            '녹사평대로11길', '왕십리로', '봉은사로113길']))]

    target_link.reset_index(drop=True, inplace=True)

    for i in (range(len(target_link))):
        t_node = target_link.iloc[i]['T_NODE']
        f_node = target_link.iloc[i]['T_NODE']
        road_name = target_link.iloc[i]['ROAD_NAME']
        matches = seoul_link_restore[
            (seoul_link_restore['F_NODE'] == t_node) &
            (~(seoul_link_restore['T_NODE'] == f_node)) &
            (seoul_link_restore['ROAD_NAME'] == road_name)]
        
        if len(matches)==0:
            matches = seoul_link_restore[
                (seoul_link_restore['F_NODE'] == t_node) &
                (~(seoul_link_restore['T_NODE'] == f_node))]

        if len(matches)==0:
            matches = seoul_link_restore[
                (seoul_link_restore['F_NODE'] == f_node) |
                (seoul_link_restore['T_NODE'] == t_node)]

        try:
            added_link = target_link.iloc[i].copy()
            matches.reset_index(drop=True, inplace=True)
            added_link['SERVICE_ID'] = matches.iloc[0]['SERVICE_ID']
            added_link['STD_ID'] = added_link['LINK_ID']

            seoul_link_restore = pd.concat([seoul_link_restore, added_link.to_frame().transpose()])
            seoul_link_restore.reset_index(drop=True, inplace=True)
        except: pass

print(f'After reconnecting the 1st multi-hop missing, the length of seoul link is {len(seoul_link_restore)}')

for iteration in range(4):
    target_link = metropolitan_link[
        (~metropolitan_link['LINK_ID'].isin(seoul_link_restore['LINK_ID'].values)) &
        (metropolitan_link['ROAD_NAME'].isin(['올림픽대로', '강변역로', '현충로', '이촌로', '이촌로2길', '청파로', '동작대로', '강변북로']))
        ]

    target_link.reset_index(drop=True, inplace=True)

    for i in (range(len(target_link))):
        t_node = target_link.iloc[i]['T_NODE']
        f_node = target_link.iloc[i]['T_NODE']
        road_name = target_link.iloc[i]['ROAD_NAME']
        matches = seoul_link_restore[
            (seoul_link_restore['F_NODE'] == t_node) &
            (~(seoul_link_restore['T_NODE'] == f_node)) &
            (seoul_link_restore['ROAD_NAME'] == road_name)]
        
        if len(matches)==0:
            matches = seoul_link_restore[
                (seoul_link_restore['F_NODE'] == t_node) &
                (~(seoul_link_restore['T_NODE'] == f_node))]

        if len(matches)==0:
            matches = seoul_link_restore[
                (seoul_link_restore['F_NODE'] == f_node) |
                (seoul_link_restore['T_NODE'] == t_node)]

        try:
            added_link = target_link.iloc[i].copy()
            matches.reset_index(drop=True, inplace=True)
            added_link['SERVICE_ID'] = matches.iloc[0]['SERVICE_ID']
            added_link['STD_ID'] = added_link['LINK_ID']

            seoul_link_restore = pd.concat([seoul_link_restore, added_link.to_frame().transpose()])
            seoul_link_restore.reset_index(drop=True, inplace=True)
        except: pass

print(f'After reconnecting the 2nd multi-hop missing, the length of seoul link is {len(seoul_link_restore)}')
del t_node, f_node, matches, added_link, i, iteration, road_name, target_link
seoul_link_restore.crs = crs

print(f'The total number of service link in seoul raw is {len(seoul_link["SERVICE_ID"].unique())}')
print(f'The total number of service link in seoul restore is {len(seoul_link_restore["SERVICE_ID"].unique())}')

### 2. 3. Save

In [ ]:
# Add restore to the end of the filename if the mode is restore

seoul_link_restore.sort_values(by=['SERVICE_ID', 'LINK_ID'], inplace=True)
seoul_link_restore.reset_index(drop=True, inplace=True)

seoul_node_list_restore = np.union1d(np.unique(seoul_link_restore['F_NODE'].values), np.unique(seoul_link_restore['T_NODE'].values))
seoul_node_restore = metropolitan_node[metropolitan_node['NODE_ID'].isin(seoul_node_list_restore)]
seoul_node_restore.sort_values(by=['NODE_ID'])
seoul_node_restore.reset_index(drop=True, inplace=True)

seoul_node_restore.to_file("seoul_node_restore.shp", encoding='utf-8')

seoul_link_restore = seoul_link_restore.apply(pd.to_numeric, errors='ignore')
seoul_link_restore.to_file("seoul_link_restore.shp", encoding='utf-8')

seoul_link_restore.to_csv('seoul_link_restore.csv', encoding='euc-kr', index=False)
seoul_node_restore.to_csv('seoul_node_restore.csv', encoding='euc-kr', index=False)

seoul_link.sort_values(by=['SERVICE_ID', 'LINK_ID'], inplace=True)
seoul_link.reset_index(drop=True, inplace=True)

seoul_node_list = np.union1d(np.unique(seoul_link['F_NODE'].values), np.unique(seoul_link['T_NODE'].values))
seoul_node = metropolitan_node[metropolitan_node['NODE_ID'].isin(seoul_node_list)]
seoul_node.sort_values(by=['NODE_ID'])
seoul_node.reset_index(drop=True, inplace=True)

seoul_node.to_file("seoul_node_raw.shp", encoding='utf-8')

seoul_link = seoul_link.apply(pd.to_numeric, errors='ignore')
seoul_link.to_file("seoul_link_raw.shp", encoding='utf-8')

seoul_link.to_csv('seoul_link_raw.csv', encoding='euc-kr', index=False)
seoul_node.to_csv('seoul_node_raw.csv', encoding='euc-kr', index=False)

del metropolitan_link, metropolitan_node, seoul_link_mapping, seoul_node_list

In [ ]:
# 이 사이에 restore 파일에서 서울시 밖으로 과도하게 확장된 링크를 자르고 그 링크를 seoul_link_restore라고 저장
# 참고로 그 서울시 파일의 좌표계는 5179이며, 손으로 직접 폴리곤을 그려서 잘라야 한다. 절묘하게 튀어나와 있는 링크들이 가끔 있기 때문

seoul_link_restore = geopandas.read_file('seoul_link_restore.shp', encoding='utf-8')

seoul_link_restore.sort_values(by=['SERVICE_ID', 'LINK_ID'], inplace=True)
seoul_link_restore.reset_index(drop=True, inplace=True)

seoul_node_restore_list = np.union1d(np.unique(seoul_link_restore['F_NODE'].values), np.unique(seoul_link_restore['T_NODE'].values))

metropolitan_node = geopandas.read_file('[2022-09-05]NODELINKDATA/metropolitan_node.shp', encoding='utf-8')
metropolitan_node = metropolitan_node.apply(pd.to_numeric, errors='ignore')
metropolitan_node.sort_values(by=['NODE_ID'], inplace=True)

seoul_node_restore = metropolitan_node[metropolitan_node['NODE_ID'].isin(seoul_node_restore_list)]
seoul_node_restore.sort_values(by=['NODE_ID'])
seoul_node_restore.reset_index(drop=True, inplace=True)

seoul_node_restore.to_file("seoul_node_restore.shp", encoding='utf-8')

seoul_link_restore = seoul_link_restore.apply(pd.to_numeric, errors='ignore')
seoul_link_restore.to_file("seoul_link_restore.shp", encoding='utf-8')

seoul_link_restore.to_csv('seoul_link_restore.csv', encoding='euc-kr', index=False)
seoul_node_restore.to_csv('seoul_node_restore.csv', encoding='euc-kr', index=False)

print(f'The total number of service link in seoul raw is {len(seoul_link["SERVICE_ID"].unique())}')
print(f'The total number of service link in seoul restore is {len(seoul_link_restore["SERVICE_ID"].unique())}')

## 3. Create speed dataset

In [ ]:
import os
from os import listdir
import itertools
from tqdm import tqdm 

import numpy as np
import pandas as pd

import geopandas

### 3. 1. Load and merge

In [ ]:
def change_data_type(df, datetime=True):
    df['PRCS_YEAR'] = df['PRCS_YEAR'].astype('int16')
    df['PRCS_MON'] = df['PRCS_MON'].astype('int8')
    df['PRCS_DAY'] = df['PRCS_DAY'].astype('int8')
    df['PRCS_HH'] = df['PRCS_HH'].astype('int8')
    df['PRCS_MIN'] = df['PRCS_MIN'].astype('int8')
    df['LINK_ID'] = df['LINK_ID'].astype('uint32')
    
    try: 
        df['PRCS_SPD'].replace(0, np.nan, inplace=True)
        df['PRCS_SPD'] = df['PRCS_SPD'].astype('float16')
    except: pass
    
    if datetime==True:
        df['PRCS_DATETIME'] = (
            df['PRCS_YEAR'].map(str)+df['PRCS_MON'].map(str).str.zfill(2)+df['PRCS_DAY'].map(str).str.zfill(2)
            +df['PRCS_HH'].map(str).str.zfill(2)+df['PRCS_MIN'].map(str).str.zfill(2))
        df['PRCS_DATETIME'] = pd.to_datetime(df['PRCS_DATETIME'],format='%Y%m%d%H%M')
    else: df['PRCS_DATETIME'] = pd.to_datetime(df['PRCS_DATETIME'])

    return df

In [ ]:
seoul_link = geopandas.read_file('seoul_link_raw.shp', encoding='utf-8')
seoul_link.sort_values(by=['SERVICE_ID', 'STD_ID'], inplace=True)
seoul_link.reset_index(drop=True, inplace=True)
service_link_list = seoul_link['SERVICE_ID'].unique()
service_link_list.sort()

for i, file in enumerate(listdir(os.getcwd()+'/seoul_speed_raw_data')):
    if i>=12:
        speed_data = pd.read_csv('./seoul_speed_raw_data/'+file)
        speed_data = speed_data[speed_data['LINK_ID'].isin(service_link_list)]
        speed_data = change_data_type(speed_data)

        if (i<13): file_concat = pd.DataFrame(columns=speed_data.columns)

        month, start_day, end_day = int(file[5:7]), int(file[7:9]), int(file[12:14])
        time_df = pd.DataFrame(list(itertools.product([2022], [month], range(start_day, end_day+1), range(0,24), range(0,60,5), service_link_list)), columns=speed_data.columns[:-2])
        time_df = change_data_type(time_df)

        time_df = pd.merge(
            time_df, speed_data[['PRCS_DATETIME', 'LINK_ID', 'PRCS_SPD']], how='left',
            left_on=['PRCS_DATETIME', 'LINK_ID'],  right_on=['PRCS_DATETIME', 'LINK_ID'])

        file_concat = pd.concat([file_concat, time_df])

        print(f'month = {file[5:7]}, day = {file[7:9]}, day = {file[12:14]}')

file_concat = change_data_type(file_concat, datetime=False)
file_concat.sort_values(by=['LINK_ID', 'PRCS_DATETIME'], inplace=True)
file_concat.reset_index(drop=True, inplace=True)
print(file_concat.dtypes)

print(file_concat['PRCS_SPD'].isna().sum())

del end_day, file, i, month, start_day, time_df, speed_data

### 3. 2. Interpolate nan values

In [ ]:
# Has nan value at October 2nd, 04:50. Interpolate by LINK_ID-PRCS_YEAR-PRCS_MON-PRCS_DAY-PRCS_HH...

print(file_concat['PRCS_SPD'].isna().sum())

for iteration in range(5):
    # Interpolate by LINK_ID - TIME
    print(f'sort values by {list([file_concat.columns.values[-3]])+list([file_concat.columns.values[-1]])}')
    file_concat.sort_values(by=['LINK_ID', 'PRCS_DATETIME'], inplace=True)
    file_concat.reset_index(drop=True, inplace=True)
    file_concat['PRCS_SPD'].interpolate(limit=12, inplace=True)
    print(file_concat['PRCS_SPD'].isna().sum())

    # Interpolate by TIME - LINK_ID
    print(f'sort values by {list([file_concat.columns.values[-1]])+list([file_concat.columns.values[-3]])}')
    file_concat.sort_values(by=['PRCS_DATETIME', 'LINK_ID'], inplace=True)
    file_concat.reset_index(drop=True, inplace=True)
    file_concat['PRCS_SPD'].interpolate(limit=4, inplace=True)
    print(file_concat['PRCS_SPD'].isna().sum())

file_concat.to_csv('seoul_speed.csv', encoding='utf-8', index=False)

del file_concat

## 4. Create adjacency matrix

In [45]:
import numpy as np
import pandas as pd

import geopandas

seoul_link_restore = geopandas.read_file('seoul_link_restore.shp', encoding='utf-8')
seoul_link_restore.sort_values(by=['SERVICE_ID', 'STD_ID'], inplace=True)
seoul_link_restore.reset_index(drop=True, inplace=True)
service_link_list = seoul_link_restore['SERVICE_ID'].unique()
service_link_list.sort()

seoul_link_restore_df = pd.DataFrame(columns=[
  'SERVICE_ID', 'F_NODE', 'T_NODE', 'NAME','LENGTH',
  'AVG_LANES', 'MIN_LANES', 'MAX_LANES', 'LANE_CHANGE'])

### 4. 1. Store link info - with restored links

In [46]:
for i, idx in enumerate(service_link_list):
  tmp_seoul_link_restore = seoul_link_restore[seoul_link_restore['SERVICE_ID'] == idx]

  # Min/Max/Average number of lanes link length for feature
  min_lanes, max_lanes = tmp_seoul_link_restore['LANES'].min(), tmp_seoul_link_restore['LANES'].max()
  link_length = round(tmp_seoul_link_restore['LENGTH'].sum(), 4)
  area = (tmp_seoul_link_restore['LANES']*tmp_seoul_link_restore['LENGTH']).sum()
  avg_lanes = round(area/link_length, 4)

  # Get a list of starting node (F NODE) ​​and ending node (T NODE) ​​of a service link
  f_node = tmp_seoul_link_restore['F_NODE'].unique()
  t_node = tmp_seoul_link_restore['T_NODE'].unique()

  f_node.sort()
  t_node.sort()

  tmp_list = [idx, f_node, t_node, tmp_seoul_link_restore.iloc[0]['ROAD_NAME'],
              link_length, avg_lanes, min_lanes, max_lanes, min_lanes!=max_lanes]

  seoul_link_restore_df.loc[i] = tmp_list

  del area, avg_lanes, f_node, i, idx, link_length, max_lanes, min_lanes, t_node, tmp_list, tmp_seoul_link_restore

seoul_link_restore_df.sort_values(by=['SERVICE_ID'], inplace=True)
seoul_link_restore_df.reset_index(drop=True, inplace=True)

### 4. 2. Create adjacency matrix - with restored links

In [47]:
# To remove the non-existing U-turn, need to subtract when (F_NODE list==T_NODE list) and (T_NODE list==F_NODE list)

from tqdm import tqdm

forward_a1 = pd.DataFrame(columns = service_link_list, index = service_link_list)
backward_a1 = pd.DataFrame(columns = service_link_list, index = service_link_list)
total_a1 = pd.DataFrame(columns = service_link_list, index = service_link_list)

seoul_link_restore_df_f_node = pd.DataFrame(columns=['F_NODE']*25, index = service_link_list)
seoul_link_restore_df_t_node = pd.DataFrame(columns=['T_NODE']*25, index = service_link_list)

for idx in forward_a1.index:
  tmp_df = seoul_link_restore_df[seoul_link_restore_df['SERVICE_ID']==idx]
  f_node_list, t_node_list = tmp_df['F_NODE'].values[0], tmp_df['T_NODE'].values[0]

  for i, f_node in enumerate(f_node_list):
    seoul_link_restore_df_f_node.loc[idx][i] = f_node
  for i, t_node in enumerate(t_node_list):
    seoul_link_restore_df_t_node.loc[idx][i] = t_node

for idx in forward_a1.index:
  tmp_df = seoul_link_restore_df[seoul_link_restore_df['SERVICE_ID']==idx]
  f_node_list, t_node_list = tmp_df['F_NODE'].values[0], tmp_df['T_NODE'].values[0]

  # Add to forward list if my t node is on f node of a specific link, and vice versa. Addition for a row has a value greater than 0 if there is an element
  forward_list = seoul_link_restore_df_f_node.index[seoul_link_restore_df_f_node['F_NODE'].isin(t_node_list).sum(axis=1)>0]
  backward_list = seoul_link_restore_df_t_node.index[seoul_link_restore_df_t_node['T_NODE'].isin(f_node_list).sum(axis=1)>0]
  total_list = np.union1d(forward_list, backward_list)

  forward_a1.loc[idx][forward_list] = 1
  backward_a1.loc[idx][backward_list] = 1
  total_a1.loc[idx][total_list] = 1

  for forward_link in forward_list:
    forward_link_f_node = seoul_link_restore_df_f_node.loc[forward_link][seoul_link_restore_df_f_node.loc[forward_link].notnull()].values
    forward_link_t_node = seoul_link_restore_df_t_node.loc[forward_link][seoul_link_restore_df_t_node.loc[forward_link].notnull()].values
    
    forward_link_f_diff = set(forward_link_f_node) - set(forward_link_t_node)
    forward_link_t_diff = set(forward_link_t_node) - set(forward_link_f_node)
    f_node_list_diff = set(f_node_list) - set(t_node_list)
    t_node_list_diff = set(t_node_list) - set(f_node_list)
    
    forward_link_f_node_t_node_list_diff = set(forward_link_f_node) - set(t_node_list)
    forward_link_t_node_f_node_list_diff = set(forward_link_t_node) - set(f_node_list)

    f_node_list_forward_link_t_node_diff = set(f_node_list) - set(forward_link_t_node)
    t_node_list_forward_link_f_node_diff = set(t_node_list) - set(forward_link_f_node)

    len_1 = (len(t_node_list_forward_link_f_node_diff)+len(f_node_list_forward_link_t_node_diff))
    len_2 = (len(forward_link_f_node_t_node_list_diff)+len(forward_link_t_node_f_node_list_diff))

    # Delete self-loop
    if forward_link == idx:
      forward_a1.loc[idx][forward_link] = 0
      backward_a1.loc[idx][forward_link] = 0
      total_a1.loc[idx][forward_link] = 0

    # Delete non-existing u-turn
    elif ((set(forward_link_t_node) == set(f_node_list)) & (set(forward_link_f_node) == set(t_node_list))):
      forward_a1.loc[idx][forward_link] = 0
      backward_a1.loc[idx][forward_link] = 0
      total_a1.loc[idx][forward_link] = 0

    elif ((forward_link_f_diff == t_node_list_diff) & (forward_link_t_diff == f_node_list_diff)):
      forward_a1.loc[idx][forward_link] = 0
      backward_a1.loc[idx][forward_link] = 0
      total_a1.loc[idx][forward_link] = 0

    elif ((len_1==0) or (len_2==0)):
      forward_a1.loc[idx][forward_link] = 0
      backward_a1.loc[idx][forward_link] = 0
      total_a1.loc[idx][forward_link] = 0

del tmp_df, f_node, f_node_list, t_node, t_node_list, idx, i, forward_list, backward_list, total_list, forward_link_f_node, forward_link_t_node, forward_link

### 4. 3. Store link info - with raw links

In [48]:
seoul_link_raw = geopandas.read_file('seoul_link_raw.shp', encoding='utf-8')
seoul_link_raw.sort_values(by=['SERVICE_ID', 'STD_ID'], inplace=True)
seoul_link_raw.reset_index(drop=True, inplace=True)
service_link_list = seoul_link_raw['SERVICE_ID'].unique()
service_link_list.sort()

seoul_link_raw_df = pd.DataFrame(columns=[
  'SERVICE_ID', 'F_NODE', 'T_NODE', 'NAME','LENGTH',
  'AVG_LANES', 'MIN_LANES', 'MAX_LANES', 'LANE_CHANGE'])

for i, idx in enumerate(service_link_list):
  tmp_seoul_link_raw = seoul_link_raw[seoul_link_raw['SERVICE_ID'] == idx]

  # Min/Max/Average number of lanes link length for feature
  min_lanes, max_lanes = tmp_seoul_link_raw['LANES'].min(), tmp_seoul_link_raw['LANES'].max()
  link_length = round(tmp_seoul_link_raw['LENGTH'].sum(), 4)
  area = (tmp_seoul_link_raw['LANES']*tmp_seoul_link_raw['LENGTH']).sum()
  avg_lanes = round(area/link_length, 4)

  # Get a list of starting node (F NODE) ​​and ending node (T NODE) ​​of a service link
  f_node = tmp_seoul_link_raw['F_NODE'].unique()
  t_node = tmp_seoul_link_raw['T_NODE'].unique()

  f_node.sort()
  t_node.sort()

  tmp_list = [idx, f_node, t_node, tmp_seoul_link_raw.iloc[0]['ROAD_NAME'],
              link_length, avg_lanes, min_lanes, max_lanes, min_lanes!=max_lanes]

  seoul_link_raw_df.loc[i] = tmp_list

  del area, avg_lanes, f_node, i, idx, link_length, max_lanes, min_lanes, t_node, tmp_list, tmp_seoul_link_raw

seoul_link_raw_df.sort_values(by=['SERVICE_ID'], inplace=True)
seoul_link_raw_df.reset_index(drop=True, inplace=True)

### 4. 4. Add adjacency matris - with raw links

In [49]:
# To remove the non-existing U-turn, need to subtract when (F_NODE list==T_NODE list) and (T_NODE list==F_NODE list)

from tqdm import tqdm

seoul_link_raw_df_f_node = pd.DataFrame(columns=['F_NODE']*25, index = service_link_list)
seoul_link_raw_df_t_node = pd.DataFrame(columns=['T_NODE']*25, index = service_link_list)

for idx in forward_a1.index:
  tmp_df = seoul_link_raw_df[seoul_link_raw_df['SERVICE_ID']==idx]
  f_node_list, t_node_list = tmp_df['F_NODE'].values[0], tmp_df['T_NODE'].values[0]

  for i, f_node in enumerate(f_node_list):
    seoul_link_raw_df_f_node.loc[idx][i] = f_node
  for i, t_node in enumerate(t_node_list):
    seoul_link_raw_df_t_node.loc[idx][i] = t_node

for idx in forward_a1.index:
  tmp_df = seoul_link_raw_df[seoul_link_raw_df['SERVICE_ID']==idx]
  f_node_list, t_node_list = tmp_df['F_NODE'].values[0], tmp_df['T_NODE'].values[0]

  # Add to forward list if my t node is on f node of a specific link, and vice versa. Addition for a row has a value greater than 0 if there is an element
  forward_list = seoul_link_raw_df_f_node.index[seoul_link_raw_df_f_node['F_NODE'].isin(t_node_list).sum(axis=1)>0]
  backward_list = seoul_link_raw_df_t_node.index[seoul_link_raw_df_t_node['T_NODE'].isin(f_node_list).sum(axis=1)>0]
  total_list = np.union1d(forward_list, backward_list)

  forward_a1.loc[idx][forward_list] = 1
  backward_a1.loc[idx][backward_list] = 1
  total_a1.loc[idx][total_list] = 1

  for forward_link in forward_list:
    forward_link_f_node = seoul_link_raw_df_f_node.loc[forward_link][seoul_link_raw_df_f_node.loc[forward_link].notnull()].values
    forward_link_t_node = seoul_link_raw_df_t_node.loc[forward_link][seoul_link_raw_df_t_node.loc[forward_link].notnull()].values
    
    forward_link_f_diff = set(forward_link_f_node) - set(forward_link_t_node)
    forward_link_t_diff = set(forward_link_t_node) - set(forward_link_f_node)
    f_node_list_diff = set(f_node_list) - set(t_node_list)
    t_node_list_diff = set(t_node_list) - set(f_node_list)

    # Delete self-loop
    if forward_link == idx:
      forward_a1.loc[idx][forward_link] = 0
      backward_a1.loc[idx][forward_link] = 0
      total_a1.loc[idx][forward_link] = 0

    # Delete non-existing u-turn
    elif ((set(forward_link_t_node) == set(f_node_list)) & (set(forward_link_f_node) == set(t_node_list))):
      forward_a1.loc[idx][forward_link] = 0
      backward_a1.loc[idx][forward_link] = 0
      total_a1.loc[idx][forward_link] = 0

    elif ((forward_link_f_diff == t_node_list_diff) & (forward_link_t_diff == f_node_list_diff)):
      forward_a1.loc[idx][forward_link] = 0
      backward_a1.loc[idx][forward_link] = 0
      total_a1.loc[idx][forward_link] = 0

del tmp_df, f_node, f_node_list, t_node, t_node_list, idx, i, forward_list, backward_list, total_list, forward_link_f_node, forward_link_t_node, forward_link

### 4. 5. Finalize adjacency matrix with turninfo

In [50]:
# Reflect turninfo
turninfo = pd.read_csv('[2022-09-05]NODELINKDATA/TURNINFO.csv', encoding='utf-8')

turninfo['ST_LINK'] = turninfo['ST_LINK'].astype('int64')
turninfo['ED_LINK'] = turninfo['ED_LINK'].astype('int64')
turninfo['TURN_TYPE'] = turninfo['TURN_TYPE'].astype('int64')

turninfo = turninfo[
    (turninfo['ST_LINK'].isin(seoul_link_restore['STD_ID'].unique())) &
    (turninfo['ED_LINK'].isin(seoul_link_restore['STD_ID'].unique()))]

turninfo.reset_index(drop=True, inplace=True)

for idx in turninfo.index:
    turninfo_tmp = turninfo.loc[idx]
    st_link = seoul_link_restore[seoul_link_restore['STD_ID']==turninfo_tmp['ST_LINK']]['SERVICE_ID'].values[0]
    ed_link = seoul_link_restore[seoul_link_restore['STD_ID']==turninfo_tmp['ED_LINK']]['SERVICE_ID'].values[0]
    turntype = turninfo_tmp['TURN_TYPE']

    if turntype in [2, 3, 101, 102, 103] :
        forward_a1.loc[st_link][ed_link] = 0
        backward_a1.loc[ed_link][st_link] = 0 
        total_a1.loc[st_link][ed_link] = 0

    elif turntype in [1, 11, 12] :
        forward_a1.loc[st_link][ed_link] = 1
        backward_a1.loc[ed_link][st_link] = 1
        total_a1.loc[st_link][ed_link] = 1
        total_a1.loc[ed_link][st_link] = 1

forward_a1 = forward_a1.fillna(0)
backward_a1 = backward_a1.fillna(0)
total_a1 = total_a1.fillna(0)

# Sort in ascending order of row and column
forward_a1.sort_index(axis = 0, inplace=True)
forward_a1.sort_index(axis = 1, inplace=True)
backward_a1.sort_index(axis = 0, inplace=True)
backward_a1.sort_index(axis = 1, inplace=True)
total_a1.sort_index(axis = 0, inplace=True)
total_a1.sort_index(axis = 1, inplace=True)

arr = forward_a1.to_numpy()
np.fill_diagonal(arr, 1)
forward_a1 = pd.DataFrame(arr, columns=forward_a1.columns, index=forward_a1.index)
arr = backward_a1.to_numpy()
np.fill_diagonal(arr, 1)
backward_a1 = pd.DataFrame(arr, columns=backward_a1.columns, index=backward_a1.index)
arr = total_a1.to_numpy()
np.fill_diagonal(arr, 1)
total_a1 = pd.DataFrame(arr, columns=total_a1.columns, index=total_a1.index)

del ed_link, st_link, turninfo_tmp, turntype, idx

### 4. 6. Save multiplication of a matrix

In [51]:
def matrix_multiplication(a_matrix, type):
    multiplier = a_matrix
    for i in range(1,10,1):
        multiplier.to_csv(f'seoul_adjacency_matrix/{type}/seoul_{type}_a{i}.csv')
        print(f'seoul_{type}_a{i}')
        if i<9:
            multiplier = multiplier@a_matrix
        else: pass

In [52]:
matrix_multiplication(forward_a1, 'forward')
matrix_multiplication(backward_a1, 'backward')
matrix_multiplication(total_a1, 'total')

seoul_forward_a1
seoul_forward_a2
seoul_forward_a3
seoul_forward_a4
seoul_forward_a5
seoul_forward_a6
seoul_forward_a7
seoul_forward_a8
seoul_forward_a9
seoul_backward_a1
seoul_backward_a2
seoul_backward_a3
seoul_backward_a4
seoul_backward_a5
seoul_backward_a6
seoul_backward_a7
seoul_backward_a8
seoul_backward_a9
seoul_total_a1
seoul_total_a2
seoul_total_a3
seoul_total_a4
seoul_total_a5
seoul_total_a6
seoul_total_a7
seoul_total_a8
seoul_total_a9


In [53]:
# This code can be used to get a appropriate value of beta
'''
TODO: find a good beta
'''
import itertools
for type, square_num in itertools.product(['forward', 'backward', 'total'], range(1,10,1)):
    a_matrix = pd.read_csv(f'seoul_adjacency_matrix/{type}/seoul_{type}_a{square_num}.csv', index_col=0)
    print(f'seoul_{type}_a{square_num}')
    print((a_matrix.max()).max())

seoul_forward_a1
1
seoul_forward_a2
7
seoul_forward_a3
24
seoul_forward_a4
94
seoul_forward_a5
412
seoul_forward_a6
1821
seoul_forward_a7
8107
seoul_forward_a8
36311
seoul_forward_a9
163404
seoul_backward_a1
1
seoul_backward_a2
7
seoul_backward_a3
24
seoul_backward_a4
94
seoul_backward_a5
412
seoul_backward_a6
1821
seoul_backward_a7
8107
seoul_backward_a8
36311
seoul_backward_a9
163404
seoul_total_a1
1
seoul_total_a2
15
seoul_total_a3
76
seoul_total_a4
586
seoul_total_a5
4325
seoul_total_a6
32814
seoul_total_a7
249253
seoul_total_a8
1910377
seoul_total_a9
14699616


### 4.7 Create shp file based on restore service link

In [ ]:
import numpy as np
import pandas as pd

import geopandas
from shapely.ops import linemerge
from shapely.ops import unary_union

seoul_link_restore = geopandas.read_file('seoul_link_restore.shp', encoding='utf-8')
seoul_link_restore.sort_values(by=['SERVICE_ID', 'STD_ID'], inplace=True)
seoul_link_restore.reset_index(drop=True, inplace=True)
service_link_list = seoul_link_restore['SERVICE_ID'].unique()
service_link_list.sort()

seoul_link_restore['LINK_ID'] = seoul_link_restore['LINK_ID'].astype('int64')
seoul_link_restore['F_NODE'] = seoul_link_restore['F_NODE'].astype('int64')
seoul_link_restore['T_NODE'] = seoul_link_restore['T_NODE'].astype('int64')

In [ ]:
%%capture

seoul_service_link_restore = geopandas.GeoDataFrame(columns=[
  'SERVICE_ID', 'F_NODE', 'T_NODE', 'NAME','LENGTH',
  'AVG_LANES', 'MIN_LANES', 'MAX_LANES', 'LANE_CHANGE', 'AREA', 'geometry'])

for i, idx in enumerate(service_link_list):
  tmp_seoul_link_restore = seoul_link_restore[seoul_link_restore['SERVICE_ID'] == idx]
  tmp_seoul_link_restore.reset_index(drop=True, inplace=True)

  # Min/max/mean lanes and length to use it as a feature
  min_lanes, max_lanes = tmp_seoul_link_restore['LANES'].min(), tmp_seoul_link_restore['LANES'].max()
  link_length = round(tmp_seoul_link_restore['LENGTH'].sum(), 4)
  area = (tmp_seoul_link_restore['LANES']*tmp_seoul_link_restore['LENGTH']).sum()
  avg_lanes = round(area/link_length, 4)

  # Get ininitial node(F NODE) and terminal node(T NODE)
  f_node = tmp_seoul_link_restore['F_NODE'].unique()
  t_node = tmp_seoul_link_restore['T_NODE'].unique()

  f_node.sort()
  t_node.sort()
  f_node = f_node.tolist()
  t_node = t_node.tolist()
  
  merged_line = unary_union(tmp_seoul_link_restore['geometry'])
  tmp_list = [idx, str(f_node)[1:-1], str(t_node)[1:-1], tmp_seoul_link_restore.iloc[0]['ROAD_NAME'],
              link_length, avg_lanes, min_lanes, max_lanes, min_lanes!=max_lanes, area, merged_line]

  seoul_service_link_restore.loc[i] = tmp_list

crs = seoul_link_restore.crs
seoul_service_link_restore.crs = crs

seoul_service_link_restore.set_geometry(col='geometry', inplace=True, drop=True)
seoul_service_link_restore = seoul_service_link_restore.apply(pd.to_numeric, errors='ignore')

seoul_service_link_restore.to_file("seoul_service_link_restore.shp", encoding='euc-kr')

### 4.8 Create shp file based on raw service link

In [6]:
import numpy as np
import pandas as pd

import geopandas
from shapely.ops import linemerge
from shapely.ops import unary_union

seoul_link_raw = geopandas.read_file('seoul_link_raw.shp', encoding='utf-8')
seoul_link_raw.sort_values(by=['SERVICE_ID', 'STD_ID'], inplace=True)
seoul_link_raw.reset_index(drop=True, inplace=True)
service_link_list = seoul_link_raw['SERVICE_ID'].unique()
service_link_list.sort()

seoul_link_raw['LINK_ID'] = seoul_link_raw['LINK_ID'].astype('int64')
seoul_link_raw['F_NODE'] = seoul_link_raw['F_NODE'].astype('int64')
seoul_link_raw['T_NODE'] = seoul_link_raw['T_NODE'].astype('int64')

In [8]:
%%capture

seoul_service_link_raw = geopandas.GeoDataFrame(columns=[
  'SERVICE_ID', 'F_NODE', 'T_NODE', 'NAME','LENGTH',
  'AVG_LANES', 'MIN_LANES', 'MAX_LANES', 'LANE_CHANGE', 'AREA', 'geometry'])

for i, idx in enumerate(service_link_list):
  tmp_seoul_link_raw = seoul_link_raw[seoul_link_raw['SERVICE_ID'] == idx]
  tmp_seoul_link_raw.reset_index(drop=True, inplace=True)

  # Min/max/mean lanes and length to use it as a feature
  min_lanes, max_lanes = tmp_seoul_link_raw['LANES'].min(), tmp_seoul_link_raw['LANES'].max()
  link_length = round(tmp_seoul_link_raw['LENGTH'].sum(), 4)
  area = (tmp_seoul_link_raw['LANES']*tmp_seoul_link_raw['LENGTH']).sum()
  avg_lanes = round(area/link_length, 4)

  # Get ininitial node(F NODE) and terminal node(T NODE)
  f_node = tmp_seoul_link_raw['F_NODE'].unique()
  t_node = tmp_seoul_link_raw['T_NODE'].unique()

  f_node.sort()
  t_node.sort()
  f_node = f_node.tolist()
  t_node = t_node.tolist()
  
  merged_line = unary_union(tmp_seoul_link_raw['geometry'])
  tmp_list = [idx, str(f_node)[1:-1], str(t_node)[1:-1], tmp_seoul_link_raw.iloc[0]['ROAD_NAME'],
              link_length, avg_lanes, min_lanes, max_lanes, min_lanes!=max_lanes, area, merged_line]

  seoul_service_link_raw.loc[i] = tmp_list

crs = seoul_link_raw.crs
seoul_service_link_raw.crs = crs

seoul_service_link_raw.set_geometry(col='geometry', inplace=True, drop=True)
seoul_service_link_raw = seoul_service_link_raw.apply(pd.to_numeric, errors='ignore')

seoul_service_link_raw.to_file("seoul_service_link_raw.shp", encoding='euc-kr')

## 5. Distance related part

### 5. 1. Get shortest path

In [1]:
import numpy as np
import pandas as pd
import geopandas

# Open link and node shapefile
seoul_link_raw = geopandas.read_file('seoul_link_raw.shp', encoding='utf-8')
seoul_node_raw = geopandas.read_file('seoul_node_raw.shp', encoding='utf-8')

# In seoul_node_raw dataframe, there is x, y coordinate of node in 'geometry' column
# Merge x and y coordinate of F_NODE and T_NODE to seoul_link_raw dataframe. It also have a suffix F_NODE and T_NODE
seoul_link_raw = seoul_link_raw.merge(seoul_node_raw[['NODE_ID', 'geometry']], left_on='F_NODE', right_on='NODE_ID', how='left', suffixes=('', '_F_NODE'))
# Drop coulmn named "NODE_ID" from seoul_link_raw dataframe
seoul_link_raw.drop(columns=['NODE_ID'], inplace=True)

seoul_link_raw = seoul_link_raw.merge(seoul_node_raw[['NODE_ID', 'geometry']], left_on='T_NODE', right_on='NODE_ID', how='left', suffixes=('', '_T_NODE'))
# Drop coulmn named "NODE_ID" from seoul_link_raw dataframe
seoul_link_raw.drop(columns=['NODE_ID'], inplace=True)

# Add a column that has mean of x and y coordinate of F_NODE and T_NODE
seoul_link_raw['x'] = seoul_link_raw.apply(lambda x: (x['geometry_F_NODE'].x + x['geometry_T_NODE'].x)/2, axis=1)
seoul_link_raw['y'] = seoul_link_raw.apply(lambda x: (x['geometry_F_NODE'].y + x['geometry_T_NODE'].y)/2, axis=1)

seoul_service_link_list = seoul_link_raw['SERVICE_ID'].unique()
seoul_service_link_list.sort()

# Group by 'SERVICE_LI' and calculate the mean coordinates for each group
mean_coords = seoul_link_raw.groupby('SERVICE_ID')[['x', 'y']].mean()
mean_coords.sort_index(inplace=True)

# Calculate the absolute differences in x and y coordinates
abs_diff = np.abs(mean_coords.values[:, None] - mean_coords.values)

# Compute the 1D and 2D distances
distance_1d = np.sum(abs_diff, axis=2) / 1000
distance_2d = np.sqrt(np.sum(abs_diff ** 2, axis=2)) / 1000

# Round the distances and convert them to DataFrames with the appropriate indices and columns
distance_1d_df = pd.DataFrame(distance_1d, index=mean_coords.index, columns=mean_coords.index).round(5)
distance_2d_df = pd.DataFrame(distance_2d, index=mean_coords.index, columns=mean_coords.index).round(5)

# Replace distance smaller than 0.1 to 0.1
distance_1d_df = distance_1d_df.where(distance_1d_df >= 0.1, 0.1)
distance_2d_df = distance_2d_df.where(distance_2d_df >= 0.1, 0.1)

# Check if the matrix is symmetric and contains inf value
print('The value should be True')
print(distance_1d_df.equals(distance_1d_df.T))
print(distance_2d_df.equals(distance_2d_df.T))

print('The value should be False')
print(np.isinf(distance_1d_df).any().any())
print(np.isinf(distance_2d_df).any().any())

distance_1d_df.to_csv('distance_1d.csv', encoding='utf-8')
distance_2d_df.to_csv('distance_2d.csv', encoding='utf-8')

# Get the reciprocal of the distance matrix
distance_1d_df = 1 / distance_1d_df
distance_2d_df = 1 / distance_2d_df

distance_1d_df.to_csv('distance_1d_a.csv', encoding='utf-8')
distance_2d_df.to_csv('distance_2d_a.csv', encoding='utf-8')

The value should be True
True
True
The value should be False
False
False


In [24]:
import numpy as np
import pandas as pd
import heapq
import time

def dijkstra(adj_matrix, dist_matrix, source):
    n = len(adj_matrix)
    visited = [False] * n
    distances = [float('inf')] * n
    distances[source] = 0
    pq = [(0, source)]

    while pq:
        (dist, current) = heapq.heappop(pq)
        if visited[current]:
            continue
        visited[current] = True

        for neighbor in range(n):
            if adj_matrix[current][neighbor] and not visited[neighbor]:
                new_dist = dist + dist_matrix[current][neighbor]
                if new_dist < distances[neighbor]:
                    distances[neighbor] = new_dist
                    heapq.heappush(pq, (new_dist, neighbor))

    return distances


def all_pairs_shortest_path(adj_matrix, dist_matrix):
    n = len(adj_matrix)
    all_pairs_sp = np.zeros((n, n))

    start_time = time.time()
    
    for source in range(n):
        all_pairs_sp[source] = dijkstra(adj_matrix, dist_matrix, source)

        # Calculate progress percentage
        progress = (source + 1) / n * 100
        
        # Calculate elapsed time and estimate time left
        elapsed_time = time.time() - start_time
        time_left = (elapsed_time / (source + 1)) * (n - source - 1)
        if n%10 == 0:
            print(f"Progress: {progress:.2f}% | Time elapsed: {elapsed_time:.2f}s | Estimated time left: {time_left:.2f}s")

    return all_pairs_sp

### 5. 2. Getting minimum distance

In [ ]:
forward_a1 = pd.read_csv('seoul_adjacency_matrix/forward/seoul_forward_a1.csv', encoding='utf-8', index_col=0)
distance_1d_df = pd.read_csv('distance_1d.csv', encoding='utf-8', index_col=['SERVICE_ID'])

all_pairs_sp_matrix = all_pairs_shortest_path(forward_a1.values, distance_1d_df.values)
print("All pairs shortest path matrix:")
print(all_pairs_sp_matrix)

all_pairs_sp_df = pd.DataFrame(all_pairs_sp_matrix, index=distance_1d_df.index, columns=distance_1d_df.columns).round(5)
all_pairs_sp_df.to_csv('seoul_adjacency_matrix/forward_sp_1d.csv', encoding='utf-8')

In [ ]:
backward_a1 = pd.read_csv('seoul_adjacency_matrix/backward/seoul_backward_a1.csv', encoding='utf-8', index_col=0)
distance_1d_df = pd.read_csv('distance_1d.csv', encoding='utf-8', index_col=['SERVICE_ID'])

all_pairs_sp_matrix = all_pairs_shortest_path(backward_a1.values, distance_1d_df.values)
print("All pairs shortest path matrix:")
print(all_pairs_sp_matrix)

all_pairs_sp_df = pd.DataFrame(all_pairs_sp_matrix, index=distance_1d_df.index, columns=distance_1d_df.columns).round(5)
all_pairs_sp_df.to_csv('seoul_adjacency_matrix/backward_sp_1d.csv', encoding='utf-8')

In [ ]:
total_a1 = pd.read_csv('seoul_adjacency_matrix/total/seoul_total_a1.csv', encoding='utf-8', index_col=0)
distance_1d_df = pd.read_csv('distance_1d.csv', encoding='utf-8', index_col=['SERVICE_ID'])

all_pairs_sp_matrix = all_pairs_shortest_path(total_a1.values, distance_1d_df.values)
print("All pairs shortest path matrix:")
print(all_pairs_sp_matrix)

all_pairs_sp_df = pd.DataFrame(all_pairs_sp_matrix, index=distance_1d_df.index, columns=distance_1d_df.columns).round(5)
all_pairs_sp_df.to_csv('seoul_adjacency_matrix/total_sp_1d.csv', encoding='utf-8')

### 5. 3. Get minimum hop matrix

In [54]:
import numpy as np
import pandas as pd

def floyd_warshall_min_hops(adj_matrix):
    n = len(adj_matrix)
    hop_matrix = adj_matrix.copy()

    for k in range(n):
        hop_matrix = np.minimum(hop_matrix, hop_matrix[:, k, np.newaxis] + hop_matrix[np.newaxis, k, :])

    return hop_matrix

In [55]:
forward_a1 = pd.read_csv('seoul_adjacency_matrix/forward/seoul_forward_a1.csv', encoding='utf-8', index_col=0)

adj_matrix = forward_a1.values

# Replace 0s with a large number (e.g., 1e9) except for the diagonal
large_number = 1e9
hop_matrix_modified = np.where(adj_matrix == 1, 1, large_number)
np.fill_diagonal(hop_matrix_modified, 0)

min_hop_matrix = floyd_warshall_min_hops(hop_matrix_modified)

print(min_hop_matrix)

min_hop_matrix = pd.DataFrame(min_hop_matrix, index=forward_a1.index, columns=forward_a1.columns)
min_hop_matrix.to_csv('seoul_adjacency_matrix/forward_min_hop.csv', encoding='utf-8')

[[ 0.  3.  7. ... 13. 13. 12.]
 [ 5.  0.  4. ... 18. 18. 17.]
 [ 8. 10.  0. ... 21. 21. 20.]
 ...
 [16. 11. 15. ...  0.  1.  3.]
 [17. 13. 17. ...  3.  0.  2.]
 [16. 12. 16. ...  1.  2.  0.]]


In [56]:
backward_a1 = pd.read_csv('seoul_adjacency_matrix/backward/seoul_backward_a1.csv', encoding='utf-8', index_col=0)

adj_matrix = backward_a1.values

# Replace 0s with a large number (e.g., 1e9) except for the diagonal
large_number = 1e9
hop_matrix_modified = np.where(adj_matrix == 1, 1, large_number)
np.fill_diagonal(hop_matrix_modified, 0)

min_hop_matrix = floyd_warshall_min_hops(hop_matrix_modified)

print(min_hop_matrix)

min_hop_matrix = pd.DataFrame(min_hop_matrix, index=backward_a1.index, columns=backward_a1.columns)
min_hop_matrix.to_csv('seoul_adjacency_matrix/backward_min_hop.csv', encoding='utf-8')

[[ 0.  5.  8. ... 16. 17. 16.]
 [ 3.  0. 10. ... 11. 13. 12.]
 [ 7.  4.  0. ... 15. 17. 16.]
 ...
 [13. 18. 21. ...  0.  3.  1.]
 [13. 18. 21. ...  1.  0.  2.]
 [12. 17. 20. ...  3.  2.  0.]]


In [57]:
total_a1 = pd.read_csv('seoul_adjacency_matrix/total/seoul_total_a1.csv', encoding='utf-8', index_col=0)

adj_matrix = total_a1.values

# Replace 0s with a large number (e.g., 1e9) except for the diagonal
large_number = 1e9
hop_matrix_modified = np.where(adj_matrix == 1, 1, large_number)
np.fill_diagonal(hop_matrix_modified, 0)

min_hop_matrix = floyd_warshall_min_hops(hop_matrix_modified)

print(min_hop_matrix)

min_hop_matrix = pd.DataFrame(min_hop_matrix, index=total_a1.index, columns=total_a1.columns)
min_hop_matrix.to_csv('seoul_adjacency_matrix/total_min_hop.csv', encoding='utf-8')

[[ 0.  3.  5. ... 11. 12. 11.]
 [ 3.  0.  4. ... 11. 12. 11.]
 [ 5.  4.  0. ... 15. 16. 15.]
 ...
 [11. 11. 15. ...  0.  1.  1.]
 [12. 12. 16. ...  1.  0.  2.]
 [11. 11. 15. ...  1.  2.  0.]]
